In [1]:
import cv2
import numpy as np

In [2]:
# 展示图片
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
# 计算单应性变换矩阵
def compute_homography(src_pts, dst_pts):
    N = src_pts.shape[0]
    
    # 构建计算矩阵
    A = np.zeros((2 * N, 9))
    for i in range(N):
        x, y = src_pts[i]
        u, v = dst_pts[i]
        A[2*i] = [x, y, 1, 0, 0, 0, -u * x, -u * y, -u]
        A[2*i+1] = [0, 0, 0, x, y, 1, -v * x, -v * y, -v]
    
    # 通过奇异值分解计算单应性变换矩阵
    _, _, vh = np.linalg.svd(A)
    H = vh[-1].reshape(3, 3)
    H = H / H[2][2]
    
    return H

In [4]:
im1 = cv2.imread('left.jpg')
im2 = cv2.imread('right.jpg')

In [5]:
# 匹配的点集
src_pts = np.array([[538, 167], [236, 316], [338, 343], [551, 267]]) # left
dst_pts = np.array([[500, 182], [191, 323], [284, 353], [516, 286]]) # right

im3 = im2.copy()
for point in dst_pts:
    im3 = cv2.circle(im3,(point[0], point[1]), 2, (0,255,0), -1)
show('im3', im3)
cv2.imwrite('point.jpg', im3)

True

In [10]:
H = compute_homography(src_pts, dst_pts)
print(H)
print('------------------------------------------------')
# 这里使用了cv2提供的接口进行对比
H, _ = cv2.findHomography(src_pts, dst_pts)
print(H)

[[ 6.74844909e-01 -4.04390976e-02  1.98081125e+01]
 [-1.05100802e-01  7.55496521e-01  6.72844817e+01]
 [-4.34076401e-04 -8.51748927e-05  1.00000000e+00]]
------------------------------------------------
[[ 6.74844909e-01 -4.04390976e-02  1.98081125e+01]
 [-1.05100802e-01  7.55496521e-01  6.72844817e+01]
 [-4.34076401e-04 -8.51748927e-05  1.00000000e+00]]


In [7]:
h, w = im2.shape[:2]

# 根据单应性变换矩阵将im2变换视角
im2_warp = cv2.warpPerspective(im2, np.linalg.inv(H), (w, h))

In [8]:
show('test', im2_warp)

In [9]:
cv2.imwrite('res1.jpg', im2_warp)

True